# День радио!

Сегодня мы создадим пользовательские интерфейсы, используя невероятно простую платформу Radio framework.

Приготовьтесь к радости!

Пожалуйста, обратите внимание: экраны ваших радиостанций могут отображаться в "темном" или "светлом" режиме в зависимости от настроек вашего компьютера.

In [ ]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [ ]:
import gradio as gr # oh yeah!

In [ ]:
# Загрузите переменные среды в файл с именем .env
# Выведите ключевые префиксы, которые помогут при любой отладке.

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Подключитесь к OpenAI, Anthropic и Google; прокомментируйте строки Claude или Google, если вы ими не пользуетесь.

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [ ]:
# Общее системное сообщение - больше никаких придирчивых враждебных ИИ!

system_message = "Вы очень полезный помощник"

In [ ]:
# Давайте превратим вызов GPT-4o-mini в простую функцию

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
# Это может указывать на "окончание обучения" или самую последнюю дату в данных обучения

message_gpt("What is today's date?")

## Время работы с пользовательским интерфейсом!

In [ ]:
# вот простая функция

def shout(text):
    print(f"Shout был вызван с помощью input {text}")
    return text.upper()

In [ ]:
shout("Привет!")

In [ ]:
# Простота использования радио. Это может отображаться в "светлом режиме" - позже я покажу вам, как это сделать в темном режиме.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# Добавление share=True означает, что к нему можно получить открытый доступ
# Более постоянный хостинг доступен на платформе Spaces от Hugging Face, о которой мы поговорим на следующей неделе
# ПРИМЕЧАНИЕ: Некоторым антивирусным программам и корпоративным брандмауэрам может не понравиться использование share=True. 
# Если вы работаете в рабочей сети, я советую пропустить этот тест.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# При добавлении в inbrowser=True автоматически открывается новое окно браузера

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Включение темного режима

Радио отображается в светлом или затемненном режиме в зависимости от настроек браузера и компьютера. Есть способ принудительно перевести радио в темный режим, но Gradio не рекомендует этого делать, поскольку это должно зависеть от предпочтений пользователя (в частности, из соображений доступности). Но если вы хотите включить темный режим для своих экранов, ниже описано, как это сделать.

In [ ]:
# Определите эту переменную, а затем передайте js=force_dark_mode при создании интерфейса

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
# Inputs and Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# А теперь - меняем функцию с "shout" на "message_gpt".

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Давайте используем Markdown
# Вам интересно, почему значение system_message имеет какое-то значение, если оно не упоминается в коде под ним?
# Я использую system_message как глобальную переменную, используемую в функции message_gpt (посмотрите сами)
# Не самая лучшая практика разработки программного обеспечения, но довольно распространенная во время исследований и разработок в лаборатории Jupiter!

system_message = "Вы - полезный помощник, который отвечает в формате markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Давайте создадим вызов, который будет передавать результаты в потоковом режиме
# Если вы хотите освежить в памяти генераторы (ключевое слово "yield"),
# # Пожалуйста, загляните в блокнот Intermediate Python в папке "Неделя 1".

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

## Minor improvement

I've made a small improvement to this code.

Previously, it had these lines:

```
for chunk in result:
  yield chunk
```

There's actually a more elegant way to achieve this (which Python people might call more 'Pythonic'):

`yield from result`

I cover this in more detail in the Intermediate Python notebook in the week1 folder - take a look if you'd like more.

## Небольшое улучшение

Я внес небольшое улучшение в этот код.

Ранее в нем были следующие строки:

```
for chunk in result:
  yield chunk
```

На самом деле есть более элегантный способ достичь этого (который люди, использующие Python, могли бы назвать более "питоническим").:

`yield from result`

Я рассказываю об этом более подробно в промежуточной тетради по Python в папке "Неделя 1" - взгляните, если хотите узнать больше.

In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

# Создание генератора рекламных проспектов компании

Теперь вы знаете, как это сделать - это просто!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Прежде чем вы прочтете следующие несколько ячеек</h2>
            <span style="color:#900;">
                Попробуйте сделать это самостоятельно - вернитесь к брошюре компании на 1-й неделе, на 5-й день и добавьте пользовательский интерфейс радиостанции в конце. Затем приходите и посмотрите на решение.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# Класс для представления веб-страницы

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
# Выражаем огромную благодарность Биллу Джи, который заметил, что в предыдущей версии была ошибка! Теперь исправлена.

system_message = "Вы - ассистент, который анализирует содержимое целевой страницы веб-сайта компании \
    и создает краткую брошюру о компании для потенциальных клиентов, инвесторов и новобранцев. \
    Ответьте на русском в формате markdown."

In [ ]:
def stream_brochure(company_name, url, model):
    prompt = f"Пожалуйста, подготовьте брошюру компании для {company_name}. Вот их целевая страница:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()